In [53]:
# https://www.tensorflow.org/datasets/keras_example

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import logging, os
import pandas as pd

logging.basicConfig(level=logging.INFO)
# logging.basicConfig(level=logging.DEBUG)

BASE_PATH = "../../../../../local_data/practice/tfds/"
DATA_PATH = "../../../../../local_data/tfds/"
OUTPUT_PATH = BASE_PATH+"mnist_simple_fit/"
os.system("mkdir -p " + OUTPUT_PATH)


(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

print(f"Number of training samples: {ds_train.cardinality()}")
print(f"Number of test samples: {ds_test.cardinality()}")

# plt.figure(figsize=(10, 10))
# for i, (image, label) in enumerate(ds_train.take(9)):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(image)
#     plt.title(int(label))
#     plt.axis("off")

INFO:absl:Load dataset info from /home/tmeng12/tensorflow_datasets/mnist/3.0.1
INFO:absl:Fields info.[citation, splits, supervised_keys, module_name] from disk and from code do not match. Keeping the one from code.
INFO:absl:Reusing dataset mnist (/home/tmeng12/tensorflow_datasets/mnist/3.0.1)


INFO:absl:Constructing tf.data.Dataset mnist for split ['train', 'test'], from /home/tmeng12/tensorflow_datasets/mnist/3.0.1


Number of training samples: 60000
Number of test samples: 10000


In [54]:
c1=tfds.as_dataframe(ds_train.take(20), ds_info)
col1 = c1['label']
# col1 = pd.DataFrame(train_ds, columns=["image","train_ds"])
print(col1)


0     4
1     1
2     0
3     7
4     8
5     1
6     2
7     7
8     1
9     6
10    6
11    4
12    7
13    7
14    3
15    3
16    7
17    9
18    9
19    1
Name: label, dtype: int64


2025-07-05 21:13:27.692370: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [55]:
batch_size=16

In [56]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
# ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [57]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(batch_size)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [58]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(batch_size, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=1,
    validation_data=ds_test,
)

3750/3750 [==============================] - 6s 2ms/step - loss: 0.3724 - sparse_categorical_accuracy: 0.8938 - val_loss: 0.2427 - val_sparse_categorical_accuracy: 0.9295


In [66]:
c1=tfds.as_dataframe(ds_train.take(5000), ds_info)
col1 = c1['label']
print(col1.shape)

pred = model.predict(ds_train)
# print("pred\n",pred)
# col2 = pd.DataFrame(pred[0:20], columns=["p0","p1","p2","p3","p4","p5","p6","p7","p8","p9"])
col2 = pd.DataFrame(pred, columns=["p0","p1","p2","p3","p4","p5","p6","p7","p8","p9"])
# print(col2)

compare = pd.concat([col1, col2], axis=1)
print(compare)
compare.to_csv(OUTPUT_PATH + "pred_train.csv", index=False)

(3750,)
3750/3750 [==============================] - 2s 660us/step
                                                  label        p0         p1  \
0      [4, 1, 0, 7, 8, 1, 2, 7, 1, 6, 6, 4, 7, 7, 3, 3] -5.956169  -2.458499   
1      [7, 9, 9, 1, 0, 6, 6, 9, 9, 4, 8, 9, 4, 7, 3, 3] -7.247459   5.286657   
2      [0, 9, 4, 9, 0, 6, 8, 4, 7, 2, 6, 0, 3, 1, 1, 7]  6.364180 -13.030648   
3      [2, 4, 4, 6, 5, 1, 9, 3, 2, 4, 3, 4, 4, 7, 5, 8] -2.080820 -11.679232   
4      [1, 1, 4, 1, 5, 3, 5, 8, 4, 1, 1, 4, 5, 3, 2, 4] -0.852570  -8.281330   
...                                                 ...       ...        ...   
59995                                               NaN -6.878458  -8.306971   
59996                                               NaN -6.511928   4.651679   
59997                                               NaN  0.543633 -11.649591   
59998                                               NaN -8.476761   6.316989   
59999                                               N

In [63]:
pred = model.predict(ds_test)
# print("pred\n",pred)
# col2 = pd.DataFrame(pred[0:20], columns=["p0","p1","p2","p3","p4","p5","p6","p7","p8","p9"])
col2 = pd.DataFrame(pred, columns=["p0","p1","p2","p3","p4","p5","p6","p7","p8","p9"])
# print(col2)

compare = pd.concat([col1, col2], axis=1)
print(compare)
compare.to_csv(OUTPUT_PATH + "pred_test.csv", index=False)

625/625 [==============================] - 0s 682us/step
                                                 label         p0         p1  \
0     [4, 1, 0, 7, 8, 1, 2, 7, 1, 6, 6, 4, 7, 7, 3, 3]  -1.607273  -8.651509   
1     [7, 9, 9, 1, 0, 6, 6, 9, 9, 4, 8, 9, 4, 7, 3, 3]  11.293907 -13.419129   
2     [0, 9, 4, 9, 0, 6, 8, 4, 7, 2, 6, 0, 3, 1, 1, 7]  -5.026979 -13.232710   
3     [2, 4, 4, 6, 5, 1, 9, 3, 2, 4, 3, 4, 4, 7, 5, 8]  -2.464123   1.259126   
4     [1, 1, 4, 1, 5, 3, 5, 8, 4, 1, 1, 4, 5, 3, 2, 4]  -4.745425  -9.406466   
...                                                ...        ...        ...   
9995                                               NaN  -1.270953  -6.688278   
9996                                               NaN  -6.967772  -5.836210   
9997                                               NaN  -1.490661 -10.125095   
9998                                               NaN   9.247137 -15.051533   
9999                                               NaN  -2.0106